### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip install semantic-link-labs

### Import the library

In [ ]:
import sempy_labs as labs
from sempy_labs import lakehouse as lake
from sempy_labs import directlake
import sempy_labs.report as rep

workspace_name = 'Demo - Learn Fabric - 02'
dataset_name = 'Profit Analysis'

### Vertipaq Analyzer

In [ ]:
labs.vertipaq_analyzer(dataset = dataset_name, workspace = workspace_name)

Export the Vertipaq Analyzer results to append to delta tables in your lakehouse.

In [ ]:
labs.vertipaq_analyzer(dataset = dataset_name, workspace = workspace_name, export = 'table')

In [ ]:
df = spark.sql("SELECT * FROM TheGreatLake02.vertipaqanalyzer_columns LIMIT 1000")
display(df)

### Best Practice Analzyer

This runs the [standard rules](https://github.com/microsoft/Analysis-Services/tree/master/BestPracticeRules) for semantic models posted on Microsoft's GitHub.

In [ ]:
labs.run_model_bpa(dataset = dataset_name, workspace = workspace_name)

This runs the Best Practice Analyzer and exports the results to the 'modelbparesults' delta table in your Fabric lakehouse.

In [ ]:
labs.run_model_bpa(dataset = dataset_name, workspace = workspace_name, export = True)

This runs the Best Practice Analyzer with the rules translated into Italian.

In [ ]:
labs.run_model_bpa(dataset = dataset_name, workspace = workspace_name, language = 'it-IT')

<div class="alert alert-block alert-info">
<b>Note:</b> For analyzing model BPA results at scale, see the Best Practice Analyzer Report notebook (link below).
</div>

[Best Practice Analyzer Notebook](https://github.com/microsoft/semantic-link-labs/blob/main/notebooks/Best%20Practice%20Analyzer%20Report.ipynb)

### Run BPA using your own best practice rules

In [ ]:
import sempy
sempy.fabric._client._utils._init_analysis_services()
import Microsoft.AnalysisServices.Tabular as TOM
import pandas as pd

dataset_name = ''
workspace_name = ''

rules = pd.DataFrame(
    [
        (
            "Performance",
            "Table",
            "Warning",
            "Rule name...",
            lambda obj, tom: tom.is_calculated_table(table_name=obj.Name),
            'Rule description...',
            '',
        ),
        (
            "Performance",
            "Column",
            "Warning",
            "Do not use floating point data types",
            lambda obj, tom: obj.DataType == TOM.DataType.Double,
            'The "Double" floating point data type should be avoided, as it can result in unpredictable roundoff errors and decreased performance in certain scenarios. Use "Int64" or "Decimal" where appropriate (but note that "Decimal" is limited to 4 digits after the decimal sign).',
        )
    ],
    columns=[
            "Category",
            "Scope",
            "Severity",
            "Rule Name",
            "Expression",
            "Description",
            "URL",
        ],
)

labs.run_model_bpa(dataset=dataset_name, workspace=workspace_name, rules=rules)

### Direct Lake

Check if any lakehouse tables will hit the [Direct Lake guardrails](https://learn.microsoft.com/power-bi/enterprise/directlake-overview#fallback).

In [ ]:
lake.get_lakehouse_tables(lakehouse = None, workspace = None, extended = True, count_rows = False)

In [ ]:
lake.get_lakehouse_tables(lakehouse = None, workspace = None, extended = True, count_rows = False, export = True)

Check if any tables in a Direct Lake semantic model will fall back to DirectQuery.

In [ ]:
directlake.check_fallback_reason(dataset = dataset_name, workspace = workspace_name)

### [OPTIMIZE](https://docs.delta.io/latest/optimizations-oss.html) your lakehouse delta tables.

In [ ]:
lake.optimize_lakehouse_tables(tables = ['', ''], lakehouse = None, workspace = None)

Refresh/reframe your Direct Lake semantic model and restore the columns which were in memory prior to the refresh.

In [ ]:
directlake.warm_direct_lake_cache_isresident(dataset = dataset_name, workspace = workspace_name)

Ensure a warm cache for your users by putting the columns of a Direct Lake semantic model into memory based on the contents of a [perspective](https://learn.microsoft.com/analysis-services/tabular-models/perspectives-ssas-tabular?view=asallproducts-allversions).

Perspectives can be created either in [Tabular Editor 3](https://github.com/TabularEditor/TabularEditor3/releases/latest) or in [Tabular Editor 2](https://github.com/TabularEditor/TabularEditor/releases/latest) using the [Perspective Editor](https://www.elegantbi.com/post/perspectiveeditor).

In [ ]:
directlake.warm_direct_lake_cache_perspective(dataset = dataset_name, workspace = workspace_name, perspective = '', add_dependencies = True)